In [1]:
from vibegrad import Tensor

In [2]:
a = Tensor(2)
b = Tensor(4)

In [3]:
c = a*b
c.backward()
c.grad

1

In [4]:
b.grad, a.grad

(2.0, 4.0)

In [5]:
c.zero_grad()
a.zero_grad()
b.zero_grad()

In [6]:
from vibegrad.nn import Linear, Sequential

In [7]:
a = Linear(1, 1)

In [8]:
import numpy as np
np.random.uniform((1,1))

array([1., 1.])

In [9]:
a(Tensor([1,1]))

Tensor(data=[[0.02833084 0.02833084]])

In [10]:
a.weight, a.bias

(Tensor(data=[[0.02833084]]), array([0.]))

In [11]:
import torch
from torch.nn import Linear as tLinear

In [12]:
a = tLinear(1,1)

In [13]:
a(torch.tensor([1.0]))

tensor([0.3143], grad_fn=<ViewBackward0>)

In [14]:
a.weight, a.bias

(Parameter containing:
 tensor([[-0.1725]], requires_grad=True),
 Parameter containing:
 tensor([0.4868], requires_grad=True))

In [15]:
model = Sequential([
    Linear(256, 300),
    Linear(300, 300),
    Linear(300, 10)
])

In [16]:
model.parameters

'Linear(256, 300, bias=True) Linear(300, 300, bias=True) Linear(300, 10, bias=True) '

In [17]:
model.total_params()

170410

In [18]:
model = torch.nn.Sequential()
model.append(torch.nn.Linear(256, 300))
model.append(torch.nn.Linear(300,300))
model.append(torch.nn.Linear(300,10))

Sequential(
  (0): Linear(in_features=256, out_features=300, bias=True)
  (1): Linear(in_features=300, out_features=300, bias=True)
  (2): Linear(in_features=300, out_features=10, bias=True)
)

In [19]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total number of parameters: {total_params}")

Total number of parameters: 170410


In [20]:
model.parameters

<bound method Module.parameters of Sequential(
  (0): Linear(in_features=256, out_features=300, bias=True)
  (1): Linear(in_features=300, out_features=300, bias=True)
  (2): Linear(in_features=300, out_features=10, bias=True)
)>